In [1]:
!pip install -q "transformers==4.44.0" "datasets==2.21.0" "peft==0.12.0" "accelerate==0.33.0" "bitsandbytes==0.43.3" "trl==0.9.6" "safetensors>=0.3.1" "gdown==4.6"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 44.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 33.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.1/315.1 kB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.8/245.8 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 48.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.6/104.6 kB 46.7 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver doe

In [2]:
from huggingface_hub import login
from google.colab import userdata

In [3]:
login(token=userdata.get('niru_hf_read'))

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [4]:
import torch
import torch.nn as nn
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

# model = "Qwen/Qwen2-7B"
model = "Qwen/Qwen2-1.5B"

tokenizer = AutoTokenizer.from_pretrained(model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token



bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model,
    quantization_config=bnb_config,
    device_map="cuda:0",
    trust_remote_code=True
)
model.config.use_cache = False

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/138 [00:00<?, ?B/s]

In [5]:
prompt = """
Below is an instruction that describes a task. Write a Python function using Pandas to accomplish the task described below.

### Instruction:
how many unique items are there

header columns with sample data:
data = {
    "invoice_date": ["2023-09-11", "2023-09-12", "2023-09-13", "2023-09-14", "2023-09-15"],
    "customer_code": ["C006", "C007", "C006", "C008", "C009"],
    "customer_name": ["Frank", "Grace", "Frank", "Hannah", "Ian"],
    "gender": ["Male", "Female", "Male", "Female", "Male"],
    "item_code": ["I005", "I004", "I003", "I002", "I001"],
    "item_name": ["Widget E", "Widget D", "Widget C", "Widget B", "Widget A"],
    "unit_price": ["$50", "$40", "$30", "$20", "$10"],
    "quantity": [2, 3, 1, 4, 5],
    "total_amount": [100, 120, 30, 80, 50],
    "tax": [10, 12, 3, 8, 5],
    "currency": ["USD", "USD", "USD", "EUR", "EUR"],
    "discount": [0.05, 0, 0, 0.1, 0.05],
    "total_discount_after": [95, 120, 30, 72, 47.5]
}

### Response:
"""
input_ids = tokenizer(prompt, return_tensors="pt").to('cuda')

outputs = model.generate(**input_ids, max_new_tokens=850)
print(tokenizer.decode(outputs[0]))

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.



Below is an instruction that describes a task. Write a Python function using Pandas to accomplish the task described below.

### Instruction:
how many unique items are there

header columns with sample data:
data = {
    "invoice_date": ["2023-09-11", "2023-09-12", "2023-09-13", "2023-09-14", "2023-09-15"],
    "customer_code": ["C006", "C007", "C006", "C008", "C009"],
    "customer_name": ["Frank", "Grace", "Frank", "Hannah", "Ian"],
    "gender": ["Male", "Female", "Male", "Female", "Male"],
    "item_code": ["I005", "I004", "I003", "I002", "I001"],
    "item_name": ["Widget E", "Widget D", "Widget C", "Widget B", "Widget A"],
    "unit_price": ["$50", "$40", "$30", "$20", "$10"],
    "quantity": [2, 3, 1, 4, 5],
    "total_amount": [100, 120, 30, 80, 50],
    "tax": [10, 12, 3, 8, 5],
    "currency": ["USD", "USD", "USD", "EUR", "EUR"],
    "discount": [0.05, 0, 0, 0.1, 0.05],
    "total_discount_after": [95, 120, 30, 72, 47.5]
}

### Response:
python
import pandas as pd

data = {


# Preparing Dataset

In [8]:
!gdown --id 1zfjg_3viiQ1IOR4AbsKiT4QCY1I1qWg8

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1zfjg_3viiQ1IOR4AbsKiT4QCY1I1qWg8
To: /content/Pandas_Code_SFT_Dataset.csv
100% 77.8k/77.8k [00:00<00:00, 3.21MB/s]


In [9]:
import pandas as pd
from datasets import Dataset

dataset_df = pd.read_csv('Pandas_Code_SFT_Dataset.csv')
df = dataset_df.sample(frac=1).reset_index(drop=True)
print(df.columns)

Index(['Dataset Name', 'Columns with Sample Data', 'Question',
       'Expected Code'],
      dtype='object')


In [10]:
df.head(5)

,Dataset Name,Columns with Sample Data,Question,Expected Code
0,sales,"data = {\n ""invoice_date"": [""2023-09-01"", ""...",What was customer C001 total sales figure for ...,```\ndef process(df):\n import datetime\n\n...
1,transaction,"data = {\n ""TransactionDate"": [""2023-01-15""...",determine the percentage of sales tax in relat...,```\ndef process(df):\n # Calculate the sal...
2,sales,"data = {\n ""invoice_date"": [""2023-09-01"", ""...",What is the total sales amount generated by cu...,```\ndef process(df):\n # Check if the cust...
3,outlet,"data = {\n ""Item_Identifier"": ['FDA15', 'DR...",How many items are categorized as low fat?,```\ndef process(df):\n # Set the preferred...
4,patient,"data = {\n ""patient_id"": [1001, 1002, 1003,...",Who are the top 5 critical patients with the m...,```\ndef process(df):\n # Filter for critic...


In [11]:
dataset = Dataset.from_pandas(df)
dataset

Dataset({
    features: ['Dataset Name', 'Columns with Sample Data', 'Question', 'Expected Code'],
    num_rows: 52
})

In [12]:
def apply_prompt_template(examples):
  instruction = examples['Question']
  context = examples["Columns with Sample Data"]
  response = examples['Expected Code']

  full_prompt = f"""Below is an instruction that describes a task. Write a Python function using Pandas to accomplish the task described below.

### Instruction:
{instruction}

header columns with sample data:
{context}

### Response:
{response}

### End
"""
  return { "text": full_prompt }

dataset = dataset.map(apply_prompt_template)
print(dataset["text"][0])

Map:   0%|          | 0/52 [00:00<?, ? examples/s]

Below is an instruction that describes a task. Write a Python function using Pandas to accomplish the task described below.

### Instruction:
What was customer C001 total sales figure for the past six months?

header columns with sample data:
data = {
    "invoice_date": ["2023-09-01", "2023-09-02", "2023-09-03", "2023-09-04", "2023-09-05"],
    "customer_code": ["C001", "C005", "C001", "C003", "C004"],
    "customer_name": ["Alice", "Bob", "Alice", "Charlie", "David"],
    "gender": ["Male", "Female", "Male", "Female", "Male"],
    "item_code": ["I001", "I002", "I003", "I004", "I005"],
    "item_name": ["Widget A", "Widget B", "Widget C", "Widget D", "Widget E"],
    "unit_price": [10, 20, 30, 40, 50],
    "quantity": [1, 2, 3, 1, 5],
    "total_amount": [10, 40, 90, 40, 250],
    "tax": [1, 4, 9, 4, 25],
    "currency": ["USD", "USD", "USD", "USD", "USD"],
    "discount": [0, 0, 0.05, 0, 0.1],
    "total_discount_after": [10, 40, 85.5, 40, 225]
}

### Response:
```
def process(df):
 

# Train

In [13]:
from peft import LoraConfig, get_peft_model

lora_alpha = 16
lora_dropout = 0.1
lora_r = 64

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=['q_proj', 'o_proj', 'gate_proj', 'up_proj', 'down_proj', 'k_proj', 'v_proj'] # Choose all linear layers from the model
)

model = get_peft_model(model, peft_config)

In [14]:
import transformers
from trl import SFTTrainer

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    dataset_text_field="text",
    peft_config=peft_config,
    max_seq_length=1024,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        max_steps=50,
        learning_rate=2e-4,
        logging_steps=1,
        output_dir="outputs",
        optim="paged_adamw_8bit",
        save_strategy="epoch",
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:280: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:318: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/52 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


Step,Training Loss
1,0.900900
2,0.917500
3,1.005800
4,0.880200
5,0.694500
6,0.815100
7,0.853400
8,0.613900
9,0.553500
10,0.708900


TrainOutput(global_step=50, training_loss=0.4252970752120018, metrics={'train_runtime': 72.0811, 'train_samples_per_second': 2.775, 'train_steps_per_second': 0.694, 'total_flos': 904303187742720.0, 'train_loss': 0.4252970752120018, 'epoch': 3.8461538461538463})

In [15]:
from transformers import pipeline

pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer)

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'JambaForCausalLM', 'JetMoeForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'Mamba2ForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausal

# Test

In [16]:
prompt = """
Below is an instruction that describes a task. Write a Python function using Pandas to accomplish the task described below.

### Instruction:
how many unique items are there

header columns with sample data:
data = {
    "invoice_date": ["2023-09-11", "2023-09-12", "2023-09-13", "2023-09-14", "2023-09-15"],
    "customer_code": ["C006", "C007", "C006", "C008", "C009"],
    "customer_name": ["Frank", "Grace", "Frank", "Hannah", "Ian"],
    "gender": ["Male", "Female", "Male", "Female", "Male"],
    "item_code": ["I005", "I004", "I003", "I002", "I001"],
    "item_name": ["Widget E", "Widget D", "Widget C", "Widget B", "Widget A"],
    "unit_price": ["$50", "$40", "$30", "$20", "$10"],
    "quantity": [2, 3, 1, 4, 5],
    "total_amount": [100, 120, 30, 80, 50],
    "tax": [10, 12, 3, 8, 5],
    "currency": ["USD", "USD", "USD", "EUR", "EUR"],
    "discount": [0.05, 0, 0, 0.1, 0.05],
    "total_discount_after": [95, 120, 30, 72, 47.5]
}

### Response:
"""


result = pipe(prompt, max_length=550)
generated_text = result[0]['generated_text']
print(generated_text.split("### End")[0])

Both `max_new_tokens` (=2048) and `max_length`(=550) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



Below is an instruction that describes a task. Write a Python function using Pandas to accomplish the task described below.

### Instruction:
how many unique items are there

header columns with sample data:
data = {
    "invoice_date": ["2023-09-11", "2023-09-12", "2023-09-13", "2023-09-14", "2023-09-15"],
    "customer_code": ["C006", "C007", "C006", "C008", "C009"],
    "customer_name": ["Frank", "Grace", "Frank", "Hannah", "Ian"],
    "gender": ["Male", "Female", "Male", "Female", "Male"],
    "item_code": ["I005", "I004", "I003", "I002", "I001"],
    "item_name": ["Widget E", "Widget D", "Widget C", "Widget B", "Widget A"],
    "unit_price": ["$50", "$40", "$30", "$20", "$10"],
    "quantity": [2, 3, 1, 4, 5],
    "total_amount": [100, 120, 30, 80, 50],
    "tax": [10, 12, 3, 8, 5],
    "currency": ["USD", "USD", "USD", "EUR", "EUR"],
    "discount": [0.05, 0, 0, 0.1, 0.05],
    "total_discount_after": [95, 120, 30, 72, 47.5]
}

### Response:
```
def process(df):
    # Check if t

# Push fine-tuned model to hub

In [17]:
from huggingface_hub import login
from google.colab import userdata

In [18]:
login(token=userdata.get('niru_hf_write'))

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [19]:
trainer.save_model("Qwen2_SFT_Finetune_Pandas_Code")

In [20]:
from peft import AutoPeftModelForCausalLM

model = AutoPeftModelForCausalLM.from_pretrained(
    "/content/Qwen2_SFT_Finetune_Pandas_Code",
    low_cpu_mem_usage=True,
    torch_dtype=torch.float16
)

# Merge LoRA and base model
merged_model = model.merge_and_unload()

In [21]:
# push merged model to the hub
merged_model.push_to_hub("Qwen2_SFT_Finetune_Pandas_Code",private=True,safe_serialization=True)
tokenizer.push_to_hub("Qwen2_SFT_Finetune_Pandas_Code",private=True)

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/nirusanan/Qwen2_SFT_Finetune_Pandas_Code/commit/e17d658a8bd23c913a033c71a11a79c37ea871e7', commit_message='Upload tokenizer', commit_description='', oid='e17d658a8bd23c913a033c71a11a79c37ea871e7', pr_url=None, pr_revision=None, pr_num=None)